In [1]:
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
import pandas as pd
from os import environ
from os import walk
from os.path import join

In [2]:
from sqlalchemy.sql import exists
from sqlalchemy import insert

In [3]:
engine = sqlalchemy.create_engine("mariadb+mariadbconnector://"\
                                  +environ.get("USER")+":"\
                                  +environ.get("PSWD")+"@127.0.0.1:3306/nba")

In [4]:
Base = declarative_base()

In [5]:
class Teams(Base):
    __tablename__ = "Teams"
    ID = sqlalchemy.Column(sqlalchemy.Integer, primary_key = True)
    Name = sqlalchemy.Column(sqlalchemy.String(length=255))

In [6]:
class Players(Base):
    __tablename__ = "Players"
    ID = sqlalchemy.Column(sqlalchemy.Integer, primary_key = True)
    Name = sqlalchemy.Column(sqlalchemy.String(length=255))

In [11]:
class Connector:
    def __init__(self):
        Base.metadata.create_all(engine)
        Session = sqlalchemy.orm.sessionmaker()
        Session.configure(bind=engine)
        self.session = Session()
        
    def addPlayers(self, df):
        
        ids = df["PLAYER_ID"]
        names = df["PLAYER"]
        for (i,p) in zip(ids,names):
            e = self.session.query(exists().where(Players.ID == i)).scalar()
            if(not e):
                player = Players(ID = i, Name = p)
                self.session.add(player)
                self.session.commit()
                
    def addTeams(self, df):
        
        ids = df["TEAM_ID"].unique()
        names = df["TEAM"].unique()
        for (i,t) in zip(ids,names):
            e = self.session.query(exists().where(Teams.ID == int(i))).scalar()
            if(not e):
                team = Teams(ID = int(i), Name = t)
                self.session.add(team)
                self.session.commit()

In [ ]:
def players_csv_to_db(c):
    for root,_, files in walk("../player_stats/"):
        for f in files:
            path = join(root,f)
            if("regular_season.csv" in path):
                df = pd.read_csv(path)
                c.addPlayers(df)

In [12]:
def team_csv_to_db(c):
    for root,_, files in walk("../player_stats/"):
        for f in files:
            path = join(root,f)
            if("regular_season.csv" in path):
                df = pd.read_csv(path)
                c.addTeams(df)

In [13]:
c = Connector()